# calculate integrated grid

In [1]:
import numpy as np
from astropy.io import fits
from speedysedfit.integrate_grid import calc_integrated_grid, get_responses, calc_comarcs_integrated_grid
from speedysedfit import model
from speedysedfit import model, reddening, filters
from astropy.io import ascii, fits
from astropy.table import Table
from multiprocessing import cpu_count, Manager, Process 

# get the SPEEDYFIT_MODELS file

In [3]:
import os
import glob
basedir=os.environ.get('SPEEDYFIT_MODELS', None)
dire = os.path.join(basedir, '..', 'transmission_curves', '*SWIFT*')
print(dire)
glob.glob(dire)

In [12]:
ebvs=np.r_[0:2.05:0.05]

In [5]:
gridfile = model.get_grid_file(integrated=False, grid=os.path.join(basedir, 'TlustyO_G_v10.fits'))
print(gridfile)

/share/lijiao/speedyfit/modelgrids/TlustyO_G_v10.fits


In [4]:
gridfile = model.get_grid_file(integrated=False, grid='kurucz')
print(gridfile)

/share/lijiao/speedyfit/modelgrids/kurucz93_z0.0_k2odfnew_sed.fits


In [5]:
gridfile = model.get_grid_file(integrated=False, grid='comarcs')
print(gridfile)

/share/lijiao/speedyfit/modelgrids/COMARCS.fits


In [6]:
responses0 = ['SWIFT', 'GALEX', 'IUE', 'STROMGREN', 'JOHNSON', 'GAIA3E', 'GAIA2', 'SKYMAPPER', 'APASS', 'SDSS', '2MASS', 'WISE', 'PANSTARRS']

In [7]:
filters.list_response(name='SWIFT')

['SWIFT.B',
 'SWIFT.U',
 'SWIFT.UVM2',
 'SWIFT.UVW1',
 'SWIFT.UVW2',
 'SWIFT.V',
 'SWIFT.white']

In [10]:
filters.eff_wave('SWIFT.UVW2')

2054.61

In [11]:
filters.list_response('SWIFT')

['SWIFT.B',
 'SWIFT.U',
 'SWIFT.UVM2',
 'SWIFT.UVW1',
 'SWIFT.UVW2',
 'SWIFT.V',
 'SWIFT.white']

In [14]:
ff = fits.open(gridfile)
data = ff[1].data
wave, flux = data['wavelength'], data['flux']
teffs = []
loggs = []
hdus = []
for hdu in ff[1:]:
    teffs.append(float(hdu.header['TEFF']))
    loggs.append(float(hdu.header['LOGG']))
    hdus.append(hdu)
responses = get_responses(responses=responses0, wave=wave)
# -- definition of one process for multi processing:

law='fitzpatrick2004'
Rv=3.1
grid='kurucz'
    
def do_ebv_process(ebvs, arr, responses):
    # Run over all reddening values, calculate the reddened model and integrate it over all photbands.
    for ebv in ebvs:
        # redden the model
        flux_ = reddening.redden(flux, wave=wave, ebv=ebv, rtype='flux', law=law, Rv=Rv)
        # calculate synthetic fluxes
        synflux = filters.synthetic_flux(wave, flux_, responses)
        # append to results
        arr.append([np.concatenate(([ebv], synflux))])
        
# -- prepare the array containing the integrated fluxes
#   (1 row per model, 1 column for each response curve and teff, logg, ebv and total luminosity)
output = np.zeros((len(teffs) * len(ebvs), 4 + len(responses)))
start = 0

manager = Manager()                                                                                                                                               
arr = manager.list([])

... subselection: SWIFT
... subselection: GALEX
... subselection: IUE
... subselection: STROMGREN
... subselection: JOHNSON
... subselection: GAIA3E
... subselection: GAIA2
... subselection: SKYMAPPER
... subselection: APASS
... subselection: SDSS
... subselection: 2MASS
... subselection: WISE
... subselection: PANSTARRS
Selected response curves: SWIFT.B, SWIFT.U, SWIFT.UVM2, SWIFT.UVW1, SWIFT.UVW2, SWIFT.V, SWIFT.white, GALEX.FUV, GALEX.NUV, IUE.B1, IUE.B2, IUE.B3, STROMGREN.B, STROMGREN.HBN, STROMGREN.HBW, STROMGREN.U, STROMGREN.V, STROMGREN.Y, JOHNSON.B, JOHNSON.H, JOHNSON.I, JOHNSON.J, JOHNSON.K, JOHNSON.L, JOHNSON.M, JOHNSON.N, JOHNSON.R, JOHNSON.U, JOHNSON.V, GAIA3E.BP, GAIA3E.G, GAIA3E.RP, GAIA2.BP, GAIA2.G, GAIA2.RP, SKYMAPPER.G, SKYMAPPER.I, SKYMAPPER.R, SKYMAPPER.U, SKYMAPPER.V, SKYMAPPER.Z, APASS.B, APASS.G, APASS.I, APASS.R, APASS.V, APASS2.B, APASS2.G, APASS2.I, APASS2.R, APASS2.V, SDSS.G, SDSS.GP, SDSS.I, SDSS.IP, SDSS.R, SDSS.RP, SDSS.U, SDSS.UP, SDSS.Z, SDSS.ZP, 2MASS.H

In [15]:
do_ebv_process(ebvs[0:1], arr, responses[0:2])

In [13]:
filters.list_response('PANSTARRS')

['PANSTARRS.G', 'PANSTARRS.I', 'PANSTARRS.R', 'PANSTARRS.Y', 'PANSTARRS.Z']

# calculate integrated grid of kurucz

In [6]:
calc_integrated_grid(threads=15, ebvs=ebvs, law='fitzpatrick2004', Rv=3.1,
                         responses=responses0, grid='kurucz') 

In [19]:
!mv ikurucz93_z0.0_k2odfnew_sed_lawfitzpatrick2004_Rv3.10.fits $SPEEDYFIT_MODELS

# calculate integrated grid of tmap

In [23]:
#calc_integrated_grid(threads=15, ebvs=ebvs, law='fitzpatrick2004', Rv=3.1,
#                         responses=responses0, grid='tmap') 

In [22]:
#!mv iTMAP2012_sdOB_extended_lawfitzpatrick2004_Rv3.10.fits $SPEEDYFIT_MODELS

# calculate integrated grid of munari

In [3]:
#calc_integrated_grid(threads=15, ebvs=ebvs, law='fitzpatrick2004', Rv=3.1,
#                         responses=responses0, grid='munari') 

In [2]:
!mv iMunari2005_extended_lawfitzpatrick2004_Rv3.10.fits $SPEEDYFIT_MODELS

# calculate intergrated grid of TLUSTYO

In [7]:
#calc_integrated_grid(threads=15, ebvs=ebvs, law='fitzpatrick2004', Rv=3.1,
#                         responses=responses0, grid='tlustyB') 

In [ ]:
!mv iTlustyB_G_v2_lawfitzpatrick2004_Rv3.10.fits $SPEEDYFIT_MODELS

# calculate integrated grid of COMARCS

In [8]:
calc_comarcs_integrated_grid(threads=30, ebvs=ebvs, law='fitzpatrick2004', Rv=3.1,
                         responses=responses0, grid='comarcs') 

... subselection: SWIFT
... subselection: GALEX
... subselection: IUE
... subselection: STROMGREN
... subselection: JOHNSON
... subselection: GAIA3E
... subselection: GAIA2
... subselection: SKYMAPPER
... subselection: APASS
... subselection: SDSS
... subselection: 2MASS
... subselection: WISE
... subselection: PANSTARRS
Selected response curves: SWIFT.B, SWIFT.U, SWIFT.UVM2, SWIFT.UVW1, SWIFT.UVW2, SWIFT.V, SWIFT.white, GALEX.FUV, GALEX.NUV, IUE.B1, IUE.B2, IUE.B3, STROMGREN.B, STROMGREN.HBN, STROMGREN.HBW, STROMGREN.U, STROMGREN.V, STROMGREN.Y, JOHNSON.B, JOHNSON.H, JOHNSON.I, JOHNSON.J, JOHNSON.K, JOHNSON.L, JOHNSON.M, JOHNSON.N, JOHNSON.R, JOHNSON.U, JOHNSON.V, GAIA3E.BP, GAIA3E.G, GAIA3E.RP, GAIA2.BP, GAIA2.G, GAIA2.RP, SKYMAPPER.G, SKYMAPPER.I, SKYMAPPER.R, SKYMAPPER.U, SKYMAPPER.V, SKYMAPPER.Z, APASS.B, APASS.G, APASS.I, APASS.R, APASS.V, APASS2.B, APASS2.G, APASS2.I, APASS2.R, APASS2.V, SDSS.G, SDSS.GP, SDSS.I, SDSS.IP, SDSS.R, SDSS.RP, SDSS.U, SDSS.UP, SDSS.Z, SDSS.ZP, 2MASS.H

'iCOMARCS_lawfitzpatrick2004_Rv3.10.fits'

In [10]:
!mv iCOMARCS_lawfitzpatrick2004_Rv3.10.fits $SPEEDYFIT_MODELS

check integrated grid of COMARCS

In [20]:
hdul = fits.open(os.path.join(basedir, 'iCOMARCS_lawfitzpatrick2004_Rv3.10.fits'))

teff,logg,OH,CH,FeH,Xi,Labs,ebv,SWIFT.B,SWIFT.U,SWIFT.UVM2,SWIFT.UVW1,SWIFT.UVW2,SWIFT.V,SWIFT.white,GALEX.FUV,GALEX.NUV,IUE.B1,IUE.B2,IUE.B3,STROMGREN.B,STROMGREN.HBN,STROMGREN.HBW,STROMGREN.U,STROMGREN.V,STROMGREN.Y,JOHNSON.B,JOHNSON.H,JOHNSON.I,JOHNSON.J,JOHNSON.K,JOHNSON.L,JOHNSON.M,JOHNSON.N,JOHNSON.R,JOHNSON.U,JOHNSON.V,GAIA3E.BP,GAIA3E.G,GAIA3E.RP,GAIA2.BP,GAIA2.G,GAIA2.RP,SKYMAPPER.G,SKYMAPPER.I,SKYMAPPER.R,SKYMAPPER.U,SKYMAPPER.V,SKYMAPPER.Z,APASS.B,APASS.G,APASS.I,APASS.R,APASS.V,APASS2.B,APASS2.G,APASS2.I,APASS2.R,APASS2.V,SDSS.G,SDSS.GP,SDSS.I,SDSS.IP,SDSS.R,SDSS.RP,SDSS.U,SDSS.UP,SDSS.Z,SDSS.ZP,2MASS.H,2MASS.J,2MASS.KS,WISE.W1,WISE.W2,WISE.W3,WISE.W4,PANSTARRS.G,PANSTARRS.I,PANSTARRS.R,PANSTARRS.Y,PANSTARRS.Z
float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
3400.0,1.5,8.8029,8.8073,7.563,2.5,0.11862223,0.0,78122.25,8905.783,6471.25,39276.0,47944.152,222703.48,132046.48,8165.688,8165.688,8165.688,8165.688,8165.688,128724.78,180957.45,170880.7,6771.2964,38791.848,239160.44,90845.77,385656.72,402106.7,406501.88,173746.7,35597.562,7811.135,672.878,349470.72,8190.8066,234845.67,205112.72,305190.03,379127.8,201356.66,305794.44,379112.25,179539.44,379398.94,302496.9,7410.3076,10963.814,422086.06,70877.9,126870.5,368998.28,315168.0,227338.66,67813.29,122633.79,368169.94,313772.1,224207.3,123611.52,135575.66,370349.9,374233.84,314665.47,315743.66,7628.6265,16773.664,409272.78,411303.62,389859.94,414587.12,184963.22,41845.855,11041.341,370.09842,31.802443,150849.75,369672.06,313772.06,371369.28,422138.3
3400.0,1.5,8.8029,8.8073,7.563,2.5,0.11862223,0.05,65294.39,7188.1553,5125.624,32845.754,39975.47,193400.17,114736.516,6455.693,6455.693,6455.693,6455.693,6455.693,108184.086,153448.97,144729.97,5396.9033,31765.06,207663.08,76395.25,377714.47,376662.34,392672.34,171508.06,35402.66,7787.3745,672.1354,317339.22,6581.9644,204769.5,179387.44,276809.9,350255.72,176039.27,277424.47,349913.2,154670.08,349744.75,268777.84,5920.8076,8886.912,397061.9,59137.586,107628.195,338617.94,280322.34,197731.92,56546.664,103941.34,337739.12,278926.6,194876.61,104808.04,115424.2,339895.78,344813.78,279817.72,281466.12,6111.432,14905.646,384328.78,392713.28,381878.97,400458.7,182620.61,41598.062,11002.59,369.80603,31.80244,128733.266,339508.2,279392.66,351529.56,394981.3
3400.0,1.5,8.8029,8.8073,7.563,2.5,0.11862223,0.1,54579.316,5803.8574,4059.8113,27484.174,33340.0,167963.44,99782.82,5103.792,5103.792,5103.792,5103.792,5103.792,90922.625,130122.92,122581.26,4301.576,26011.41,180316.36,64256.66,369937.16,352874.03,379327.3,169298.86,35208.855,7763.6885,671.39386,288246.34,5289.398,178575.78,156965.47,251281.33,323667.16,153982.19,251906.27,323043.7,133289.48,322424.56,238862.72,4730.8394,7203.776,373536.34,49347.902,91320.805,310754.88,249354.27,172004.1,47157.76,88114.625,309839.6,247976.02,169405.73,88881.56,98290.3,311961.78,317727.7,248854.34,250942.44,4896.1904,13348.893,360919.34,375006.84,374062.6,386816.06,180307.86,41351.77,10963.978,369.514,31.802439,109881.39,311820.88,248813.4,332753.84,369577.3
3400.0,1.5,8.8029,8.8073,7.563,2.5,0.11862223,0.15,45627.96,4687.8613,3215.6255,23011.117,27813.129,145881.64,86853.06,4034.9956,4034.9956,4034.9956,4034.9956,4034.9956,76416.625,110343.3,103822.12,3428.6323,21300.172,156572.97,54057.977,362321.4,330631.1,366449.3,167118.69,35016.133,7740.0767,670.6533,261896.81,4250.8804,155759.55,137411.92,228294.95,299

In [ ]:
tab = Table(hdul[1].data)
tab